In [176]:
# Import libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from collections import defaultdict

url = "https://gtworld.de/en/collections/skin-care-en"

try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all elements with class "card-wrapper"
    grid_items = soup.find_all(class_="card-wrapper")

    data = []
    for item in grid_items:
        elements = {}
        # Find specific tags within each grid item
        for tag in ['a', 'div', 'span', 'h3', 'h4', 'p', 'source']:
            found_tags = item.find_all(tag)
            if found_tags:
                elements[tag] = [t.get_text(strip=True) for t in found_tags]

        if elements:
            data.append(elements)

    if data:
        dff = pd.DataFrame(data)
        print("Scraped Data:")
        print(dff.to_markdown(index=False)) # Use to_markdown for a table format
    else:
        print("No relevant elements found within the 'grid__item' containers.")

except requests.exceptions.Timeout:
    print("Error fetching the URL: Request timed out.")
except requests.exceptions.ConnectionError as e:
    print(f"Connection error occurred: {e}")
    print("The server might be down, your network is having issues, or the website is blocking scraping.")
except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Scraped Data:
| div                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | h3                    | a                                                                                                                                                                    | span                                                                                                                                 

In [177]:
datadiv = dff['div'].to_frame()
datadiv = datadiv.iloc[7:]
print(datadiv)

                                                  div
7   [, Haz NatSkin Papaya Face Cream 100mlSaleswom...
8   [, American Dream OUD Spicy & Sensual Body Cre...
9   [, Topsygel Hand Cream Lightening Anti-TachesS...
10  [, Parachute Coconut Oil Jar 200ml \ 500mlSale...
11  [, NHH Batana Butter With Rosemary For Hair & ...
12  [, NHH Batana Butter Authentic For Hair & Skin...
13  [, HAZ NATSkin Papaya Body Oil 180mlSaleswoman...
14  [, Ebony Baby Jelly Cocoa 440mlSaleswoman:Ebon...
15  [, Ebony Baby Jelly Argan Oil 440mlSaleswoman:...
16  [, Ebony Baby Jelly Rosemary 440mlSaleswoman:E...
17  [, Savon bel'Eunice Carotte / Papaye 600gSales...
18  [, Sta-Sof-Fro Powder Hair Dye - Dark BrownSal...
19  [, Magno Shower Gel 550ml / 600mlSaleswoman:Ma...
20  [, Gazzela SoapSaleswoman:GazzelaGazzela SoapR...
21  [, Vaseline intensive care coconut restore bod...
22  [, Secret d'Afrique Skin Lightening Carrot Gly...
23  [, B White Turmeric with Saffron Face Cream 50...
24  [, Heatol Moisturizing S

In [178]:
# Create a new DataFrame so that incse I encounter an error, I can come back and rerun from here
df_extracted_info = datadiv.copy()

# Function to extract product name, brand name, and price
def extract_product_info(div_list):
    product_name = None
    brand_name = None
    price = None

    if div_list:
        div_content = "".join(div_list)

        # Extract Product Name (everything to the left of 'Saleswoman')
        product_name_match = re.search(r'(.*?)Saleswoman', div_content)
        if product_name_match:
            product_name = product_name_match.group(1).strip()

        # Extract Brand Name (everything between 'Saleswoman:' and the product name)
        if product_name:
             brand_name_match = re.search(r'Saleswoman:(.*?)\s*' + re.escape(product_name), div_content)
             if brand_name_match:
                 brand_name = brand_name_match.group(1).strip()


        # Extract Price (everything between ', Selling price' and the next 2 digits after comma)
        price_match = re.search(r'Selling price\s*(.*?,.{2})', div_content)
        if price_match:
            price = price_match.group(1).strip()


    return pd.Series([product_name, brand_name, price])

# Apply the extraction function to the 'div' column and create new columns
df_extracted_info[['product name', 'brand name', 'price']] = df_extracted_info['div'].apply(extract_product_info)
df_extracted_info.insert(0, 'Product ID', range(1, 1 + len(df_extracted_info)))


# Drop the original 'div' column
df_extracted_info = df_extracted_info.drop(columns=['div'])


# Print the new DataFrame
print("\nDataFrame with extracted information:")
print(df_extracted_info.to_markdown(index=False))



DataFrame with extracted information:
|   Product ID | product name                                                                 | brand name             | price   |
|-------------:|:-----------------------------------------------------------------------------|:-----------------------|:--------|
|            1 | Haz NatSkin Papaya Face Cream 100ml                                          | HAZ                    | €6,99   |
|            2 | American Dream OUD Spicy & Sensual Body Cream with Cocoa & Shea Butter 500ml | American Dream         | €9,99   |
|            3 | Topsygel Hand Cream Lightening Anti-Taches                                   | Topsygel               | €14,99  |
|            4 | Parachute Coconut Oil Jar 200ml \ 500ml                                      | Parachute              | €7,99   |
|            5 | NHH Batana Butter With Rosemary For Hair & Skin                              | Natural Health Harmony | €11,99  |
|            6 | NHH Batana Butter Authentic

In [179]:
# Creating product website urls
df_extracted_info['Product Website'] = 'https://gtworld.de/en/products/' + \
                                     df_extracted_info['product name'].str.lower() \
                                     .str.replace(' & ', ' ') \
                                     .str.replace('/', '-') \
                                     .str.replace('\\', '-') \
                                     .str.replace(' ', '-') \
                                     .str.replace("---","-")\
                                     .str.replace("'", "")


print("\nDataFrame with extracted information and Product Website:")
print(df_extracted_info.to_markdown(index=False))


DataFrame with extracted information and Product Website:
|   Product ID | product name                                                                 | brand name             | price   | Product Website                                                                                         |
|-------------:|:-----------------------------------------------------------------------------|:-----------------------|:--------|:--------------------------------------------------------------------------------------------------------|
|            1 | Haz NatSkin Papaya Face Cream 100ml                                          | HAZ                    | €6,99   | https://gtworld.de/en/products/haz-natskin-papaya-face-cream-100ml                                      |
|            2 | American Dream OUD Spicy & Sensual Body Cream with Cocoa & Shea Butter 500ml | American Dream         | €9,99   | https://gtworld.de/en/products/american-dream-oud-spicy-sensual-body-cream-with-cocoa-shea-butter-5

In [180]:
# Extracting image urls and product website URL
def extract_product_website_info(product_id, product_url):
    try:
        response = requests.get(product_url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        img_urls = []
        img_tags = soup.find_all('img')
        for img in img_tags:
            if 'src' in img.attrs:
                img_urls.append(img['src'])
            if 'data-src' in img.attrs: # Handle lazy loading
                img_urls.append(img['data-src'])

        source_tags = soup.find_all('source')
        for source in source_tags:
             if 'srcset' in source.attrs:
                 # srcset can have multiple URLs and descriptors, just take the first URL
                 urls = source['srcset'].split(',')[0].split(' ')[0]
                 img_urls.append(urls)

        return {'Product ID': product_id, 'Product Website': product_url, 'image_urls': img_urls if img_urls else None}

    except requests.exceptions.Timeout:
        print(f"Error fetching {product_url}: Request timed out.")
        return {'Product ID': product_id, 'Product Website': product_url, 'image_urls': None}
    except requests.exceptions.RequestException as e:
        print(f"An error occurred fetching {product_url}: {e}")
        return {'Product ID': product_id, 'Product Website': product_url, 'image_urls': None}
    except Exception as e:
        print(f"An unexpected error occurred fetching {product_url}: {e}")
        return {'Product ID': product_id, 'Product Website': product_url, 'image_urls': None}

product_website_data = []

# Iterate through the 'Product Website' column of the existing DataFrame
if 'Product Website' in df_extracted_info.columns:
    for index, row in df_extracted_info.iterrows():
        product_id = row['Product ID'] # Get the Product ID
        product_url = row['Product Website'] # Get the product URL
        extracted_info = extract_product_website_info(product_id, product_url)
        product_website_data.append(extracted_info)

# Create a new DataFrame from the collected data
df_product_website_info = pd.DataFrame(product_website_data)

# Merge the new DataFrame with the existing df_extracted_info on 'Product ID'
df_extracted_info = pd.merge(df_extracted_info, df_product_website_info, on='Product ID', how='left')

# Print the new DataFrame
print("\nDataFrame with extracted information and Image URLs:")
if 'image_urls' in df_extracted_info.columns:
    print(df_extracted_info.to_markdown(index=False))
else:
     print("Image URLs column not added. Check extraction process.")
     print(df_extracted_info.to_markdown(index=False))

An error occurred fetching https://gtworld.de/en/products/nhh-batana-butter-authentic-for-hair-skin: 404 Client Error: Not Found for url: https://gtworld.de/en/products/nhh-batana-butter-authentic-for-hair-skin
An error occurred fetching https://gtworld.de/en/products/haz-natskin-papaya-body-oil-180ml: 404 Client Error: Not Found for url: https://gtworld.de/en/products/haz-natskin-papaya-body-oil-180ml
An error occurred fetching https://gtworld.de/en/products/ebony-baby-jelly-cocoa-440ml: 404 Client Error: Not Found for url: https://gtworld.de/en/products/ebony-baby-jelly-cocoa-440ml
An error occurred fetching https://gtworld.de/en/products/savon-beleunice-carotte-papaye-600g: 404 Client Error: Not Found for url: https://gtworld.de/en/products/savon-beleunice-carotte-papaye-600g
An error occurred fetching https://gtworld.de/en/products/magno-shower-gel-550ml-600ml: 404 Client Error: Not Found for url: https://gtworld.de/en/products/magno-shower-gel-550ml-600ml
An error occurred fetchin

In [181]:
# Filter rows where 'image_urls' contains a URL with '1500' to have images of good quality
df_filtered_1500 = df_extracted_info[
    df_extracted_info['image_urls'].apply(
        lambda urls: any('1500' in str(url) for url in urls) if isinstance(urls, list) else False
    )
].copy()

# Function to find the first URL containing '1500' in a list
def find_first_1500_url(urls):
    if isinstance(urls, list):
        for url in urls:
            if '1500' in str(url):
                if not str(url).startswith('https://') and not str(url).startswith('http://'):
                    return f'https:{url}'
                return url
    return None

# Apply the function to create the new 'first_1500_image_url' column
df_filtered_1500['first_1500_image_url'] = df_filtered_1500['image_urls'].apply(find_first_1500_url)

# Select only the desired columns for the new table, using the correct column name for Product Website
df_1500_urls_table = df_filtered_1500[['Product ID', 'product name', 'brand name', 'price', 'Product Website_x', 'first_1500_image_url']]

# Rename 'Product Website_x' to 'Product Website' for clarity in the final table
df_1500_urls_table = df_1500_urls_table.rename(columns={'Product Website_x': 'Product Website'})

# Print the new filtered table
print("\nFiltered table with first image URL containing '1500':")
print(df_1500_urls_table.to_markdown(index=False))


Filtered table with first image URL containing '1500':
|   Product ID | product name                                                                 | brand name             | price   | Product Website                                                                                         | first_1500_image_url                                                                                                               |
|-------------:|:-----------------------------------------------------------------------------|:-----------------------|:--------|:--------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------|
|            1 | Haz NatSkin Papaya Face Cream 100ml                                          | HAZ                    | €6,99   | https://gtworld.de/en/products/haz-natskin-papaya-face-cream-100ml       

In [182]:
# for each website, extract span whose class=barcode_value and span with class=metafield-multi_line_text_field as ingredients

def extract_barcode_ingredients(product_url):
    try:
        response = requests.get(product_url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        extracted_data = {}

        # Extract span with class "barcode_value"
        barcode_span = soup.find('span', class_='barcode-value')
        if barcode_span:
            extracted_data['product code'] = barcode_span.get_text(strip=True)

        # Extract span with class "metafield-multi_line_text_field"
        ingredients_span = soup.find('span', class_='metafield-multi_line_text_field')
        if ingredients_span:
            extracted_data['ingredients'] = ingredients_span.get_text(strip=True)

        return extracted_data

    except requests.exceptions.Timeout:
        print(f"Error fetching {product_url}: Request timed out.")
        return {}
    except requests.exceptions.RequestException as e:
        print(f"An error occurred fetching {product_url}: {e}")
        return {}
    except Exception as e:
        print(f"An unexpected error occurred fetching {product_url}: {e}")
        return {}

# Apply the new extraction function to the 'Product Website' column of the filtered table
if 'Product Website' in df_1500_urls_table.columns:
    extracted_barcode_ingredients_data = df_1500_urls_table['Product Website'].apply(extract_barcode_ingredients)

    # Convert the Series of dictionaries into a DataFrame
    df_barcode_ingredients = pd.DataFrame(list(extracted_barcode_ingredients_data))

    # Concatenate the new columns to the existing df_1500_urls_table
    # Ensure the indices align
    df_1500_urls_table = pd.concat([df_1500_urls_table.reset_index(drop=True), df_barcode_ingredients.reset_index(drop=True)], axis=1)

# Print the updated df_1500_urls_table
print("\nFiltered table with barcode and ingredients:")
print(df_1500_urls_table.to_markdown(index=False))


Filtered table with barcode and ingredients:
|   Product ID | product name                                                                 | brand name             | price   | Product Website                                                                                         | first_1500_image_url                                                                                                               |   product code | ingredients                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [183]:
# Drop missing values in product code
df_1500_urls_table = df_1500_urls_table.dropna(subset=['product code'])

print("\nFiltered table after removing rows with NaN in 'product code':")
print(df_1500_urls_table.to_markdown(index=False))


Filtered table after removing rows with NaN in 'product code':
|   Product ID | product name                                                                 | brand name       | price   | Product Website                                                                                         | first_1500_image_url                                                                                                               |   product code | ingredients                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [184]:
# Gather all the ingrediants that are seperated by commas into one list and use them as you reference base to find similar ones

# Function to split ingredients string into a list and clean up
def split_and_clean_ingredients(ingredients_str):
    if isinstance(ingredients_str, str):
        # Split by comma and clean up whitespace and potential extra characters
        ingredients = [re.sub(r'[.]$', '', ingredient.strip()) for ingredient in ingredients_str.split(',') if ingredient.strip()]
        return ingredients
    return []

# Apply the split_and_clean_ingredients function to create a new column with lists of ingredients
df_1500_urls_table['ingredients_list'] = df_1500_urls_table['ingredients'].apply(split_and_clean_ingredients)

# Create a master list of all unique individual ingredients
all_ingredients = set()
for ingredients_list in df_1500_urls_table['ingredients_list']:
    for ingredient in ingredients_list:
        all_ingredients.add(ingredient)

# Convert the set back to a list
all_ingredients_list = list(all_ingredients)


# Create a dictionary to store products by the individual ingredients they contain
ingredient_presence_in_products = defaultdict(list)

for index, row in df_1500_urls_table.iterrows():
    product_id = row['Product ID']
    ingredients_list = row['ingredients_list']
    # Check for the presence of each ingredient from the master list in the product's ingredients string
    product_ingredients_string = " ".join(ingredients_list).lower() # Convert to lower for case-insensitive matching

    for ingredient in all_ingredients_list:
        if ingredient.lower() in product_ingredients_string:
             ingredient_presence_in_products[ingredient].append(product_id)


# Identify pairs of products that share individual ingredients (at least two)
shared_ingredient_pairs_new_approach = set()

for ingredient, product_ids in ingredient_presence_in_products.items():
    if len(product_ids) > 1:
        for i in range(len(product_ids)):
            for j in range(i + 1, len(product_ids)):
                pair = tuple(sorted((product_ids[i], product_ids[j])))
                shared_ingredient_pairs_new_approach.add(pair)

# Create a dictionary to store shared ingredients (based on presence) for each product pair and the count
shared_ingredients_details_new_approach = {}

for pair in shared_ingredient_pairs_new_approach:
    product_id1, product_id2 = pair
    # Get the full ingredients lists for these two products
    ingredients_list1 = df_1500_urls_table[df_1500_urls_table['Product ID'] == product_id1]['ingredients_list'].iloc[0]
    ingredients_list2 = df_1500_urls_table[df_1500_urls_table['Product ID'] == product_id2]['ingredients_list'].iloc[0]

    # Find common ingredients based on presence in the original lists
    common_ingredients = [ingredient for ingredient in all_ingredients_list if ingredient.lower() in " ".join(ingredients_list1).lower() and ingredient.lower() in " ".join(ingredients_list2).lower()]

    if len(common_ingredients) >= 2:
         shared_ingredients_details_new_approach[pair] = {
             'common_ingredients': common_ingredients,
             'num_shared': len(common_ingredients)
         }

# Prepare data for a new DataFrame for shared products details
shared_products_data_new_approach = []
for (product_id1, product_id2), details in shared_ingredients_details_new_approach.items():
    # Get product names for easier identification
    product_name1 = df_1500_urls_table[df_1500_urls_table['Product ID'] == product_id1]['product name'].iloc[0]
    product_name2 = df_1500_urls_table[df_1500_urls_table['Product ID'] == product_id2]['product name'].iloc[0]
    shared_products_data_new_approach.append({
        'Product ID 1': product_id1,
        'Product Name 1': product_name1,
        'Product ID 2': product_id2,
        'Product Name 2': product_name2,
        'Number of Shared Ingredients (Presence)': details['num_shared'],
        'Shared Ingredients (Presence)': ", ".join(details['common_ingredients'])
    })

# Create a DataFrame of products that share two or more ingredients (based on presence)
df_shared_products_new_approach = pd.DataFrame(shared_products_data_new_approach)

# Sort by the number of shared ingredients (descending) only if the DataFrame is not empty
if not df_shared_products_new_approach.empty:
    df_shared_products_new_approach = df_shared_products_new_approach.sort_values(by='Number of Shared Ingredients (Presence)', ascending=False)

# Print the table of products sharing two or more ingredients (based on presence)
print("\nProducts Sharing Two or More Ingredients (Based on Presence):")
if not df_shared_products_new_approach.empty:
    print(df_shared_products_new_approach.to_markdown(index=False))
else:
    print("No products found sharing two or more ingredients based on presence.")

# Update the original dataframe with the 'Similar Products' column based on the new approach
similar_products_map_new_approach = defaultdict(list)
shared_ingredients_with_similar_map = defaultdict(list) # New dictionary to store shared ingredients with similar products

if not df_shared_products_new_approach.empty:
    for index, row in df_shared_products_new_approach.iterrows():
        pid1 = row['Product ID 1']
        pid2 = row['Product ID 2']
        shared_ingredients_str = row['Shared Ingredients (Presence)']

        similar_products_map_new_approach[pid1].append(f"{pid2} ({row['Number of Shared Ingredients (Presence)']} shared)")
        similar_products_map_new_approach[pid2].append(f"{pid1} ({row['Number of Shared Ingredients (Presence)']} shared)")

        # Store shared ingredients for each product with its similar product(s)
        shared_ingredients_with_similar_map[pid1].append(shared_ingredients_str)
        shared_ingredients_with_similar_map[pid2].append(shared_ingredients_str)


df_1500_urls_table['Similar Products (>=2 shared ingredients - Presence)'] = df_1500_urls_table['Product ID'].apply(
    lambda pid: ", ".join(similar_products_map_new_approach[pid]) if pid in similar_products_map_new_approach else "None"
)

# Add the new column for listing shared ingredients with similar products
df_1500_urls_table['Shared Ingredients with Similar Products'] = df_1500_urls_table['Product ID'].apply(
    lambda pid: "; ".join(shared_ingredients_with_similar_map[pid]) if pid in shared_ingredients_with_similar_map else "None"
)


# Print the updated original dataframe with the new columns
print("\nOriginal DataFrame with Similar Products and Shared Ingredients Columns:")
print(df_1500_urls_table[['Product ID', 'product name', 'ingredients', 'Similar Products (>=2 shared ingredients - Presence)', 'Shared Ingredients with Similar Products']].to_markdown(index=False))


Products Sharing Two or More Ingredients (Based on Presence):
|   Product ID 1 | Product Name 1                             |   Product ID 2 | Product Name 2                      |   Number of Shared Ingredients (Presence) | Shared Ingredients (Presence)                                               |
|---------------:|:-------------------------------------------|---------------:|:------------------------------------|------------------------------------------:|:----------------------------------------------------------------------------|
|              3 | Topsygel Hand Cream Lightening Anti-Taches |             19 | Skala Maracuja e Oleo Pataua 1000ml |                                         5 | Cetearyl Alcohol, Stearamidopropyl Dimethylamine, Citric Acid, Aqua, Parfum |

Original DataFrame with Similar Products and Shared Ingredients Columns:
|   Product ID | product name                                                                 | ingredients                                 

In [185]:
# Drop unnecessary columns
df_1500_urls_table.drop(columns=['ingredients_list','Similar Products (>=2 shared ingredients - Presence)'], inplace=True)

# Extract website name
df_1500_urls_table['Website Name'] = df_1500_urls_table['Product Website'].apply(lambda x: x.replace('https://', '').split('/')[0] if isinstance(x, str) else x)


# Change and arrange column names appropriately
df_1500_urls_table.rename(columns={
    'Product ID': 'Product ID',
    'brand name': 'Brand Name',
    'product name': 'Product Name',
    'first_1500_image_url': 'Product Image URL',
    'product code': 'Barcode',
    'ingredients': 'Ingredients',
    'price': 'Price (€)',
    'Product Website': 'Product Website',
    'Website Name': 'Website Name',
    'Shared Ingredients with Similar Products': 'Shared Ingredients with Similar Products'
}, inplace=True)

# Reorder columns to match the requested order
new_column_order = [
    'Product ID',
    'Brand Name',
    'Product Name',
    'Product Image URL',
    'Barcode',
    'Ingredients',
    'Price (€)',
    'Website Name',
    'Product Website',
    'Shared Ingredients with Similar Products'
]

# Ensure all desired columns exist before reordering
existing_columns = df_1500_urls_table.columns.tolist()
final_column_order = [col for col in new_column_order if col in existing_columns]

df_1500_urls_table = df_1500_urls_table[final_column_order]
df_1500_urls_table.to_csv('Scraped Product Data.csv', index=False)
df_1500_urls_table

,Product ID,Brand Name,Product Name,Product Image URL,Barcode,Ingredients,Price (€),Website Name,Product Website,Shared Ingredients with Similar Products
0,1,HAZ,Haz NatSkin Papaya Face Cream 100ml,https://gtworld.de/cdn/shop/files/haz-natskin-...,5060433981210,NaN,"€6,99",gtworld.de,https://gtworld.de/en/products/haz-natskin-pap...,None
1,2,American Dream,American Dream OUD Spicy & Sensual Body Cream ...,https://gtworld.de/cdn/shop/files/g.jpg?v=1747...,5035620044429,NaN,"€9,99",gtworld.de,https://gtworld.de/en/products/american-dream-...,None
2,3,Topsygel,Topsygel Hand Cream Lightening Anti-Taches,https://gtworld.de/cdn/shop/files/902770_L.jpg...,3440570261055,AQUA / WATER PARAFFINUM LIQUIDUM / MINERAL OIL...,"€14,99",gtworld.de,https://gtworld.de/en/products/topsygel-hand-c...,"Cetearyl Alcohol, Stearamidopropyl Dimethylami..."
3,4,Parachute,Parachute Coconut Oil Jar 200ml \ 500ml,https://gtworld.de/cdn/shop/files/parachute-co...,8901088058216,NaN,"€7,99",gtworld.de,https://gtworld.de/en/products/parachute-cocon...,None
6,10,Ebony Baby,Ebony Baby Jelly Rosemary 440ml,https://gtworld.de/cdn/shop/files/ebony-baby-j...,4053743793865,"Petrolatum, Rosemary, Fragrance (perfume).","€5,99",gtworld.de,https://gtworld.de/en/products/ebony-baby-jell...,None
7,12,Sta-Sof-Fro,Sta-Sof-Fro Powder Hair Dye - Dark Brown,https://gtworld.de/cdn/shop/files/sta-sof-fro-...,5017857040627,NaN,"€5,99",gtworld.de,https://gtworld.de/en/products/sta-sof-fro-pow...,None
8,14,Gazzela,Gazzela Soap,https://gtworld.de/cdn/shop/files/gazzela-soap...,4053743799959,NaN,"€3,99",gtworld.de,https://gtworld.de/en/products/gazzela-soap,None
9,15,Vaseline,Vaseline intensive care coconut restore body o...,https://gtworld.de/cdn/shop/files/vaseline-int...,8886467042560,NaN,"€14,99",gtworld.de,https://gtworld.de/en/products/vaseline-intens...,None
10,16,Secret d'Afrique,Secret d'Afrique Skin Lightening Carrot Glycer...,https://gtworld.de/cdn/shop/files/secret-dafri...,8717931606042,NaN,"€9,99",gtworld.de,https://gtworld.de/en/products/secret-dafrique...,None
11,17,B White,B White Turmeric with Saffron Face Cream 50g,https://gtworld.de/cdn/shop/files/b-white-curc...,3701495900659,NaN,"€7,99",gtworld.de,https://gtworld.de/en/products/b-white-turmeri...,None
